In [25]:
import utils
import importlib

importlib.reload(utils)

gpu_ids = [4, 5]
num_gpus = len(gpu_ids)
utils.set_gpu_ids(gpu_ids)

os.environ["CUDA_VISIBLE_DEVICES"] = 5


In [2]:
import os
import regex as re
import subprocess
import multiprocessing as mp
import torch

In [3]:
# input
train_models_dirpath = "/data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf"

# output: utils.best_of_n_results_jsonl_path

In [4]:
model_variant_paths = []
for name in os.listdir(train_models_dirpath):
    path = os.path.join(train_models_dirpath, name)
    if not os.path.isdir(path):
        continue
    if not utils.is_zero3_parameters(path):
        continue
    model_variant_paths.append(path)


def extract_step_or_epoch_num(path):
    return int(re.search(r".*?(?:step|epoch)_([0-9]+)", path).group(1))


model_variant_paths.sort(key=extract_step_or_epoch_num, reverse=True)

model_variant_paths = [
    model_variant_path
    for model_variant_path in model_variant_paths
    # if 1 < extract_step_or_epoch_num(model_variant_path) < 100
    # or extract_step_or_epoch_num(model_variant_path) >= 100
]

print(model_variant_paths)

['/data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf/step_2000', '/data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf/step_1900', '/data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf/step_1800', '/data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf/step_1700', '/data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf/step_1600', '/data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf/step_1500', '/data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf/step_1400', '/data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf/step_1300', '/data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf/step_1200', '/data/users/zhang

In [6]:
for model_variant_path in model_variant_paths:
    eval_process_results = subprocess.run(
        args=[
            utils.python_path,
            "/data/users/zhangjunlei/tyx/reward-by-prm800k/src/eval-one-model-with-best-of-n.py",
            "--model_name_or_path",
            model_variant_path,
            "--gpu_ids",
            ",".join([str(gpu_id) for gpu_id in gpu_ids]),
        ]
    )
    if eval_process_results.returncode != 0:
        raise RuntimeError(f"Failed to evaluate model variant {model_variant_path}.")

os.environ["CUDA_VISIBLE_DEVICES"] = 5
ZeRO-3 model weights detected at /data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf/step_2000.
ZeRO-3 model weights converted to HF model weights at /data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf/step_2000_fp16_hf
len(prompts) = 83944
prompts[0] = A $90^\circ$ rotation around the origin in the counter-clockwise direction is applied to $7 + 2i.$  What is the resulting complex number?
I know that multiplying a complex number by $i$ rotates it $90^\circ$ counter-clockwise in the complex plane, so I can use that to find the answer.
INFO 08-24 01:40:16 llm_engine.py:70] Initializing an LLM engine with config: model='/data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf/step_2000_fp16_hf', tokenizer='/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--hf-internal-testing--llama-tokenizer/snapshots/99eceeba6e8289b

KeyboardInterrupt: 

In [ ]:
# def process_fn(gpu_id, func, *args, **kwargs):
#     torch.cuda.set_device(gpu_id)
#     func(*args, **kwargs)


# # if __name__ == '__main__':
# with mp.Pool(num_gpus) as pool:
#     pool.map(process_fn, gpu_ids)

In [24]:
# model_variant_path = "/data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf/step_2000"

# utils.eval_model_with_best_of_n(
#     model_name_or_path=model_variant_path,
#     metrics=[metric for metric in utils.all_metrics if metric != "majority_voting"],
#     debug_for={"resume_vllm_outputs": True},
# )

ZeRO-3 model weights detected at /data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf/step_2000.
ZeRO-3 model weights converted to HF model weights at /data/users/zhangjunlei/tyx/reward-by-prm800k/models/direct-prediction/meta-llama/Llama-2-7b-hf/step_2000_fp16_hf
Loading /data/users/zhangjunlei/tyx/reward-by-prm800k/datasets/problem-solution-hierarchical-samples.pkl...
Loaded
Extracting 16 subsamples...
Extracted
Resuming vLLM outputs from /data/users/zhangjunlei/tyx/reward-by-prm800k/tmp/vllm-outputs.pkl
Resumed
len(rating2prob_list) = 82595
rating2prob_list[0] = {'1': 0.6280793064277469, '-1': 0.3635044258009806, '0': 0.008416267771272466}


IndexError: list index out of range